In [ ]:
#Run this on first execution to pull deepseek-coder:1.3b from Ollama

# !ollama pull deepseek-coder:1.3b

In [5]:
from operator import itemgetter
import pandas as pd

import re
import sqlparse
from sqlparse.sql import Identifier, IdentifierList
from sqlparse.tokens import Keyword, DML

from langchain_community.llms import Ollama

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.example_selectors import BaseExampleSelector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains import create_sql_query_chain
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import os
from openai import AzureOpenAI

In [ ]:
# Loading the Questions

all_qa = load_dataset("cardiffnlp/databench", name="qa", split="train")
input_df = pd.DataFrame(all_qa)
input_df = input_df[['question', 'dataset']]

In [2]:
# Populate this with LLM you want to use from ['gpt-4o-mini', 'gpt-4o', 'deepseek-coder:1.3b']

llm_to_use = 'gpt-4o-mini'

In [ ]:
# Examples to load into Vector Database for Few-Shot selection

examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT * FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT * FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT * FROM products WHERE quantityInStock < 7000;"
    },
    {
        "input": "What is the price of `1968 Ford Mustang`?",
        "query": "SELECT * FROM products WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

examples += [
    {
        "input": "List the top 5 ranks of billionaires who are not self-made.",
        "query": "SELECT * FROM billionaires WHERE selfMade = 'No' ORDER BY rank ASC LIMIT 5;"
    },
    {
        "input": "Are there any rental properties that can accommodate more than 10 guests?",
        "query": "SELECT * FROM rental_properties WHERE maxGuests > 10;"
    },
    {
        "input": "What are the ages of the 3 oldest customers?",
        "query": "SELECT * FROM customers ORDER BY age DESC LIMIT 3;"
    },
    {
        "input": "Find all employees hired after January 1, 2020.",
        "query": "SELECT * FROM employees WHERE hireDate > '2020-01-01';"
    },
    {
        "input": "Get the average salary of employees in the 'Marketing' department.",
        "query": "SELECT * FROM employees WHERE department = 'Marketing';"
    },
    {
        "input": "Show the total sales for each product category.",
        "query": "SELECT * FROM sales GROUP BY productCategory;"
    },
    {
        "input": "List all orders placed in December 2023.",
        "query": "SELECT * FROM orders WHERE orderDate BETWEEN '2023-12-01' AND '2023-12-31';"
    },
    {
        "input": "Retrieve the names of customers who haven't placed any orders.",
        "query": "SELECT * FROM customers WHERE customerID NOT IN (SELECT DISTINCT customerID FROM orders);"
    },
    {
        "input": "Find the total revenue generated by each salesperson.",
        "query": "SELECT * FROM orders GROUP BY salespersonID;"
    },
    {
        "input": "Show the details of properties rented more than 50 times.",
        "query": "SELECT * FROM rental_properties WHERE rentalCount > 50;"
    },
    {
        "input": "Get the names and phone numbers of customers who live in New York or Los Angeles.",
        "query": "SELECT * FROM customers WHERE city IN ('New York', 'Los Angeles');"
    },
    {
        "input": "List the product names and their prices for products priced over $500.",
        "query": "SELECT * FROM products WHERE buyPrice > 500;"
    },
    {
        "input": "How many customers are there in each country?",
        "query": "SELECT * FROM customers GROUP BY country;"
    },
    {
        "input": "Retrieve the details of employees whose last name starts with 'S'.",
        "query": "SELECT * FROM employees WHERE lastName LIKE 'S%';"
    },
    {
        "input": "What is the highest-rated product in the database?",
        "query": "SELECT * FROM products ORDER BY rating DESC LIMIT 1;"
    },
    {
        "input": "Find the total number of orders and their combined value for each customer.",
        "query": "SELECT * FROM orders GROUP BY customerID;"
    },
    {
        "input": "List all cars available for rent with a daily price below $50.",
        "query": "SELECT * FROM rental_cars WHERE dailyPrice < 50;"
    },
    {
        "input": "Retrieve the total quantity of all products sold.",
        "query": "SELECT * FROM order_details;"
    },
    {
        "input": "Show the names of customers who placed more than 5 orders.",
        "query": "SELECT * FROM customers JOIN orders ON customers.customerID = orders.customerID GROUP BY customers.customerID HAVING COUNT(orders.orderID) > 5;"
    },
    {
        "input": "What is the total number of employees in each department?",
        "query": "SELECT * FROM employees GROUP BY department;"
    }
]


In [ ]:
#Setup LLM

if 'deepseek' in llm_to_use:
    llm = Ollama(model=llm_to_use, temperature = 0.01)
else:
    llm = AzureChatOpenAI(api_key='*****', azure_endpoint='*****', azure_deployment='*****', api_version='*****', model=llm_to_use, temperature=0.0)

In [ ]:
#SETUP DB and Helper Function

db = SQLDatabase.from_uri(f"sqlite:///semeval.db")
execute_query = QuerySQLDataBaseTool(db=db)

In [6]:
def extract_table_and_columns(sql_query):
# Parse the SQL query
    parsed = sqlparse.parse(sql_query)
    if not parsed:
        return None, None
   
    # Consider the first parsed statement
    statement = parsed[0]
   
    # Ensure it starts with a SELECT statement
    if not statement.get_type() == 'SELECT':
        return None, None
   
    table_name = None
    columns = []
    from_seen = False
    select_seen = False
   
    # Traverse through the tokens
    for token in statement.tokens:
        # Check for SELECT clause
        if token.ttype is DML and token.value.upper() == "SELECT":
            select_seen = True
        elif select_seen and isinstance(token, IdentifierList):
            # Capture columns from the SELECT clause
            columns = [id.get_real_name() or id.get_alias() for id in token.get_identifiers()]
            select_seen = False  # After capturing, reset the flag
        elif select_seen and isinstance(token, Identifier):
            # Single column in SELECT
            columns.append(token.get_real_name() or token.get_alias())
            select_seen = False  # Reset the flag
 
        # Check for FROM clause
        if token.ttype is Keyword and token.value.upper() == "FROM":
            from_seen = True
        elif from_seen:
            # Capture the table name after FROM
            if isinstance(token, Identifier):
                table_name = token.get_real_name()
                break
            elif isinstance(token, IdentifierList):
                # Handle multiple tables, for example in JOINs
                table_name = [id.get_real_name() for id in token.get_identifiers()]
                break
   
    return table_name, columns



In [ ]:
def create_dynamic_prompt(inputs):
    """Generate the prompt dynamically using extracted columns."""
    table_name, columns = extract_table_and_columns(inputs["query"])  
    column_headers = ", ".join(columns) 

    # Construct the prompt dynamically
    prompt = f"""Given the following user question and row(s) containing the answer, infer and answer the user question in exactly the format expected. You are also given columns headers for the table from which the row is extracted for context.
Answer only the user question directly with the information from the SQL rows given to you. Answers should strictly contain only the value expected, NOTHING ELSE. Ensure you respond only with values directly from the rows, do not write full sentences. 
The following answer formats are expected based on the question asked:
Boolean: Valid answers include True/False. If a question expects a yes/no answer, respond strictly only with True or False.
Category: A value from a cell (or a substring of a cell) in the dataset.
Number: A numerical value from a cell in the dataset, which may represent a computed statistic (e.g., average, maximum, minimum).
List: A list containing a fixed number of categories or numbers. The expected format is: "['cat', 'dog']". 
Columns available in the dataset: {column_headers}
Question: {inputs['question']}
SQL Result: {inputs['result']}
Answer:"""
    return prompt

rephrase_answer = (
    RunnableLambda(create_dynamic_prompt) |  
    llm |  
    StrOutputParser()  
)

In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)

In [9]:
vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    AzureOpenAIEmbeddings(api_key='d556174a27b043f18e8fbf6ba741a372', azure_endpoint='https://pb-open-ai-us-north-central.openai.azure.com', azure_deployment='text-embedding-ada-002', api_version='2023-03-15-preview', model="text-embedding-ada-002"),
    vectorstore,
    k=2,
    input_keys=["input"],
)

In [10]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
example_prompt=example_prompt,
example_selector=example_selector,
input_variables=["input","top_k"],
)

In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a PostgresSQL expert. Given an input question, create a syntactically correct PostgresSQL query to run. Unless otherwise specified.\n\nHere is the relevant table info: {table_info}\n\nMost columns have intuitve names.
        Return the entire row(s) that contain the final answer in context of the original question based strictly on the sql table you are given (always use SELECT (*)). Every question will be answered only from the table provided, no other source of data.

        \nBelow are a number of examples of questions and their corresponding PostgresSQL queries."""),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

generate_query = create_sql_query_chain(llm, db, final_prompt, k=500)

In [12]:
def clean_sql_query(text: str) -> str:

    block_pattern = r"```(?:sql|SQL|SQLQuery|mysql|postgresql)?\s*(.*?)\s*```"
    text = re.sub(block_pattern, r"\1", text, flags=re.DOTALL)

    prefix_pattern = r"^(?:SQL\s*Query|SQLQuery|MySQL|PostgreSQL|SQL)\s*:\s*"
    text = re.sub(prefix_pattern, "", text, flags=re.IGNORECASE)

    sql_statement_pattern = r"(SELECT.*?;)"
    sql_match = re.search(sql_statement_pattern, text, flags=re.IGNORECASE | re.DOTALL)
    if sql_match:
        text = sql_match.group(1)

    text = re.sub(r'`([^`]*)`', r'\1', text)
    text = re.sub(r'\s+', ' ', text)

    keywords = ['SELECT', 'FROM', 'WHERE', 'GROUP BY', 'HAVING', 'ORDER BY', 'LIMIT', 'JOIN', 'LEFT JOIN', 'RIGHT JOIN', 'INNER JOIN', 'OUTER JOIN', 'UNION', 'VALUES', 'INSERT', 'UPDATE', 'DELETE']
    pattern = '|'.join(r'\b{}\b'.format(k) for k in keywords)
    text = re.sub(f'({pattern})', r'\n\1', text, flags=re.IGNORECASE)

    text = text.strip()
    text = re.sub(r'\n\s*\n', '\n', text)


    return text

In [13]:

chain = (
    RunnablePassthrough.assign(
        table_names_to_use=itemgetter("dataset")  # Directly map the table details
    ) |
    RunnablePassthrough.assign(
        query=generate_query | RunnableLambda(clean_sql_query)
    ).assign(
        result=itemgetter("query") | execute_query
    )
    .assign(captured_query=RunnableLambda(lambda inputs: {"query": inputs["query"]}))
    .assign(answer=rephrase_answer)
)

In [ ]:
import concurrent.futures
from tqdm import tqdm

# Function to process a single row
def process_row(i, row):
    try:
        output = chain.invoke({
            "question": row['question'], 
            "dataset": [row['dataset']]
        })
        return i, output  # Return index to update DataFrame correctly
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        return i, None  # Return None in case of error

# Use ThreadPoolExecutor for parallel execution
with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
    # Submit tasks for each row in the DataFrame
    future_to_index = {executor.submit(process_row, i, input_df.iloc[i]): i for i in range(len(input_df))}
    
    # Initialize tqdm progress bar
    with tqdm(total=len(input_df), desc="Processing", unit="row") as pbar:
        for future in concurrent.futures.as_completed(future_to_index):  # No timeout here
            i = future_to_index[future]  # Get the corresponding row index
            try:
                # Apply per-task timeout
                i, output = future.result(timeout=10)
                if output is not None:
                    input_df.loc[i, 'Result'] = output['answer']
                    input_df.loc[i, 'sql-query'] = output['query']
                else:
                    input_df.loc[i, 'Result'] = None
                    input_df.loc[i, 'sql-query'] = None
            except concurrent.futures.TimeoutError:
                print(f"Timeout for row {i}")
                input_df.loc[i, 'Result'] = None
                input_df.loc[i, 'sql-query'] = None
            except Exception as e:
                print(f"Error processing row {i}: {e}")
                input_df.loc[i, 'Result'] = None
                input_df.loc[i, 'sql-query'] = None

            pbar.update(1)  # Update progress bar
            

In [ ]:
os.environ["AZURE_OPENAI_KEY"] = "****"
os.environ["AZURE_OPENAI_ENDPOINT"] = "****"
os.environ["AZURE_API_VERSION"] = "****"
os.environ["AZURE_DEPLOYMENT_ID"] = "****"
model_name = "gpt-4o"

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("AZURE_API_VERSION"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_ID")
)

In [ ]:
system_eval = """You are an expert in evaluating answers to questions based on format validity and relevance to the question. Your goal is not to fact-check the answers but to ensure:

The answer matches the appropriate format for the given question. Accepted formats are:
- Boolean (True/False)
- A single number/word/date
- A single list of either categories or numbers

The answer is relevant to the question. A response should only be classified as Redo if:
- The format does not match any of the acceptable formats at all (e.g., incomplete, inconsistent, or entirely different from the required format).
- The answer is completely unrelated to the question or does not attempt to address it.

If the answer aligns with the acceptable format and is reasonably related to the question, classify it as Pass. Use Pass for borderline cases unless there is a clear and significant violation of format or relevance.

You will analyze the given question and answer and determine one of the following outputs: Pass if the answer is appropriate based on the given conditions and Redo if it is not.

Strictly return only Pass or Redo, nothing else in your answer.

"""

user_eval = """Given the following question and answer, evaluate the answer based on its format and relevance:

Question: {question}
Answer: {result}

Classify the answer as either Pass or Redo.
"""

In [ ]:
# Re-Run using different chain setup in case it failed in first iteration

input_df['eval'] = ""
for i in tqdm(range(input_df.shape[0])):

    dynamic_user = user_eval.format(
    question = input_df.loc[i, 'question'],
    result = input_df.loc[i, 'Result']
    )


    messages = [
        {"role": "system", "content": system_eval},
        {"role": "user", "content": dynamic_user}
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    input_df.loc[i,'eval'] = response.choices[0].message.content

In [ ]:
df = input_df.copy()

In [ ]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
     'input':"what is price of `1968 Ford Mustang`",
     "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

examples += [
    {
        "input": "List the top 5 ranks of billionaires who are not self-made.",
        "query": "SELECT name, rank FROM billionaires WHERE selfMade = 'No' ORDER BY rank ASC LIMIT 5;"
    },
    {
        "input": "Are there any rental properties that can accommodate more than 10 guests?",
        "query": "SELECT propertyID, propertyName FROM rental_properties WHERE maxGuests > 10;"
    },
    {
        "input": "What are the ages of the 3 oldest customers?",
        "query": "SELECT name, age FROM customers ORDER BY age DESC LIMIT 3;"
    },
    {
        "input": "Find all employees hired after January 1, 2020.",
        "query": "SELECT employeeID, firstName, lastName, hireDate FROM employees WHERE hireDate > '2020-01-01';"
    },
    {
        "input": "Get the average salary of employees in the 'Marketing' department.",
        "query": "SELECT AVG(salary) AS averageSalary FROM employees WHERE department = 'Marketing';"
    },
    {
        "input": "Show the total sales for each product category.",
        "query": "SELECT productCategory, SUM(salesAmount) AS totalSales FROM sales GROUP BY productCategory;"
    },
    {
        "input": "List all orders placed in December 2023.",
        "query": "SELECT orderID, customerID, orderDate FROM orders WHERE orderDate BETWEEN '2023-12-01' AND '2023-12-31';"
    },
    {
        "input": "Retrieve the names of customers who haven't placed any orders.",
        "query": "SELECT name FROM customers WHERE customerID NOT IN (SELECT DISTINCT customerID FROM orders);"
    },
    {
        "input": "Find the total revenue generated by each salesperson.",
        "query": "SELECT salespersonID, SUM(orderAmount) AS totalRevenue FROM orders GROUP BY salespersonID;"
    },
    {
        "input": "Show the details of properties rented more than 50 times.",
        "query": "SELECT propertyID, propertyName, rentalCount FROM rental_properties WHERE rentalCount > 50;"
    },
    {
        "input": "Get the names and phone numbers of customers who live in New York or Los Angeles.",
        "query": "SELECT name, phoneNumber FROM customers WHERE city IN ('New York', 'Los Angeles');"
    },
    {
        "input": "List the product names and their prices for products priced over $500.",
        "query": "SELECT productName, buyPrice FROM products WHERE buyPrice > 500;"
    },
    {
        "input": "How many customers are there in each country?",
        "query": "SELECT country, COUNT(customerID) AS customerCount FROM customers GROUP BY country;"
    },
    {
        "input": "Retrieve the details of employees whose last name starts with 'S'.",
        "query": "SELECT employeeID, firstName, lastName FROM employees WHERE lastName LIKE 'S%';"
    },
    {
        "input": "What is the highest-rated product in the database?",
        "query": "SELECT productName, MAX(rating) AS highestRating FROM products;"
    },
    {
        "input": "Find the total number of orders and their combined value for each customer.",
        "query": "SELECT customerID, COUNT(orderID) AS totalOrders, SUM(orderAmount) AS totalValue FROM orders GROUP BY customerID;"
    },
    {
        "input": "List all cars available for rent with a daily price below $50.",
        "query": "SELECT carID, carModel, dailyPrice FROM rental_cars WHERE dailyPrice < 50;"
    },
    {
        "input": "Retrieve the total quantity of all products sold.",
        "query": "SELECT SUM(quantityOrdered) AS totalQuantity FROM order_details;"
    },
    {
        "input": "Show the names of customers who placed more than 5 orders.",
        "query": "SELECT customers.name FROM customers JOIN orders ON customers.customerID = orders.customerID GROUP BY customers.name HAVING COUNT(orders.orderID) > 5;"
    },
    {
        "input": "What is the total number of employees in each department?",
        "query": "SELECT department, COUNT(employeeID) AS employeeCount FROM employees GROUP BY department;"
    }
]


In [ ]:
vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    AzureOpenAIEmbeddings(api_key='****', azure_endpoint='****', azure_deployment='****', api_version='****', model="text-embedding-ada-002"),
    vectorstore,
    k=2,
    input_keys=["input"],
)

In [ ]:
answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query and SQL result containing the answer, infer and answer the user question in exactly the format expected. 
Answer only the user question directly with the information from the SQL rows given to you. Answers should strictly contain only the value expected, NOTHING ELSE. Ensure you respond only with values directly from the rows, do not write full sentences. 
Instructions:
- Return exactly as many values as the user question asks for from the sql result you are given. For example, if you are asked for the 4 highest ages, and they are all 80, return [80, 80, 80, 80].
- The SQL result may contain values from which the answer can be inferred. In such cases, understand what the SQL query has searched for and ensure you answer the user question directly in context of the SQL result. For example, if the question asks whether anyone under the age of 18 survived the titanic, and the sql query searches for a count of the number of people below the age of 18 who survived, your answer should be true if the sql result is a non zero value.
- Answer only with the value asked for in the question. For example if you are asked to list the rank of a person, your answer should be [rank], not their name. Do not include uneccesary information in your answer.
- Provide the answer strictly in the expected format based on the type of question asked. Ensure your answer is in the python formatted version of the respective data type:
Boolean: If the question expects a yes/no answer, respond with either True or False.
Category: Return the exact value from a cell (or a substring of a cell) in the dataset.
Number: Provide a numerical value from a cell, including computed statistics (e.g., average, maximum, minimum). Truncate to two decimals if required.
List: Return a list of categories or numbers from the relevant rows in the format: ['value1', 'value2'].
Use only the values provided in the SQL result. Do not add any external information.
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)
rephrase_answer = answer_prompt | llm | StrOutputParser()

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
example_prompt=example_prompt,
example_selector=example_selector,
input_variables=["input","top_k"],
)

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. \n\nHere is the relevant table info: {table_info}\n\n Most columns have intuitive names. Do not create filters irrelevant to the query, strictly generate the query based on what you are asked for. Ensure your query is generated exactly to answer the user question directly with no additional information.
        Return the value requested in context of the original question based strictly on the sql table you are given. Every question will be answered only from the table provided, no other source of data. Make sure the number of values returned by the SQL query are exactly as many as required in the user question.

        \nBelow are a number of examples of questions and their corresponding SQLite queries."""),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

generate_query = create_sql_query_chain(llm, db, final_prompt, k=500)

In [ ]:
chain = (
    RunnablePassthrough.assign(
        table_names_to_use=itemgetter("dataset")
    ) |
    RunnablePassthrough.assign(
        query=generate_query | RunnableLambda(clean_sql_query)
    ).assign(
        result=itemgetter("query") | execute_query
    )
    .assign(captured_query=RunnableLambda(lambda inputs: {"query": inputs["query"]}))
    .assign(answer=rephrase_answer)
)

In [ ]:
from tqdm import tqdm

for i in tqdm(range(df.shape[0])):
    if df.loc[i, 'eval'] == 'Redo':
        try:
            output = chain.invoke({
                "question": df.loc[i, 'question'], 
                "dataset": [df.loc[i, 'dataset']]
            })
            df.loc[i, 'Result'] = output['answer']
        except Exception as e:
            print(f"Skipping iteration {i} due to error: {e}")
        continue

In [27]:
df.to_excel('gpt_4o_semeval_results.xlsx')